In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import torch.optim as optim

In [17]:
import glob
import random 
import time

import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets

from torch.autograd import Variable
from torch.utils import data
import torch.nn as nn
import torch.nn.functional as F

In [18]:
transform = transforms.Compose([transforms.ToTensor()])

In [19]:
#dataset = datasets.ImageFolder('./rps-cv-images/', transform=transform) # This is all data load at once

In [20]:
train = datasets.ImageFolder('./data/train/', transform=transform)# This is train only
test = datasets.ImageFolder('./data/test/', transform=transform)# This is test only

trainloader = torch.utils.data.DataLoader(train, batch_size=8, shuffle=True)
testloader = torch.utils.data.DataLoader(test, batch_size=8, shuffle=True)

In [21]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.shortcut = nn.Sequential()

        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [22]:
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes,
                               kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [23]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        #out = self.linear(out)
        return out

In [24]:
# These are predefined resnet types
def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])

def ResNet34():
    return ResNet(BasicBlock, [3,4,6,3])

def ResNet50():
    return ResNet(Bottleneck, [3,4,6,3])

def ResNet101():
    return ResNet(Bottleneck, [3,4,23,3])

def ResNet152():
    return ResNet(Bottleneck, [3,8,36,3])

In [25]:
#net = ResNet18() # Change this to switch resnet type
torch.cuda.empty_cache()
net = torch.hub.load('pytorch/vision:v0.9.0','resnet18', pretrained=True)


if torch.cuda.is_available():
    net.cuda()

Using cache found in C:\Users\GiriGiri_Yomi/.cache\torch\hub\pytorch_vision_v0.9.0


In [26]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.cuda()
        labels = labels.cuda()
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the test images: 0 %


In [27]:
num_ftrs = net.fc.in_features
net.fc = nn.Linear(num_ftrs, 3)
if torch.cuda.is_available():
    net.cuda()

In [28]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.cuda()
        labels = labels.cuda()
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the test images: 27 %


In [29]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.000001) # 已经调好的参数

for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()

        # zero the parameter gradients
        #optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

            # print statistics
        running_loss += loss.item()
        if i % 50 == 49:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                 (epoch + 1, i + 1, running_loss / 50))
            running_loss = 0.0

print('Finished Training')

[1,    50] loss: 1.285
[1,   100] loss: 1.205
[1,   150] loss: 1.104
[1,   200] loss: 1.039
[2,    50] loss: 0.922
[2,   100] loss: 0.798
[2,   150] loss: 0.679
[2,   200] loss: 0.574
[3,    50] loss: 0.449
[3,   100] loss: 0.424
[3,   150] loss: 0.370
[3,   200] loss: 0.287
[4,    50] loss: 0.187
[4,   100] loss: 0.162
[4,   150] loss: 0.129
[4,   200] loss: 0.153
[5,    50] loss: 0.143
[5,   100] loss: 0.199
[5,   150] loss: 0.120
[5,   200] loss: 0.111
[6,    50] loss: 0.145
[6,   100] loss: 0.085
[6,   150] loss: 0.084
[6,   200] loss: 0.073
[7,    50] loss: 0.078
[7,   100] loss: 0.124
[7,   150] loss: 0.038
[7,   200] loss: 0.069
[8,    50] loss: 0.032
[8,   100] loss: 0.106
[8,   150] loss: 0.137
[8,   200] loss: 0.083
[9,    50] loss: 0.039
[9,   100] loss: 0.167
[9,   150] loss: 0.092
[9,   200] loss: 0.124
[10,    50] loss: 0.050
[10,   100] loss: 0.043
[10,   150] loss: 0.045
[10,   200] loss: 0.025
Finished Training


In [30]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.cuda()
        labels = labels.cuda()
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the test images: 98 %
